In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import umap

data_dir = '../data'

/Users/charlie/virtualenvs/vpaud/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/charlie/virtualenvs/vpaud/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/charlie/virtualenvs/vpaud/lib/python3.9/site-packages/umap/distances.

In [2]:
df = pd.read_csv(data_dir + '/pd_speech_features.csv')
display(df)
print(df.shape)

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,250,0,0.80903,0.56355,0.28385,417,416,0.004627,0.000052,0.00064,...,3.0706,3.0190,3.1212,2.4921,3.5844,3.5400,3.3805,3.2003,6.8671,0
752,250,0,0.16084,0.56499,0.59194,415,413,0.004550,0.000220,0.00143,...,1.9704,1.7451,1.8277,2.4976,5.2981,4.2616,6.3042,10.9058,28.4170,0
753,251,0,0.88389,0.72335,0.46815,381,380,0.005069,0.000103,0.00076,...,51.5607,44.4641,26.1586,6.3076,2.8601,2.5361,3.5377,3.3545,5.0424,0
754,251,0,0.83782,0.74890,0.49823,340,339,0.005679,0.000055,0.00092,...,19.1607,12.8312,8.9434,2.2044,1.9496,1.9664,2.6801,2.8332,3.7131,0


(756, 755)


In [3]:
# find all the columns which are full of zeros and remove them

zero_tol = 1e-5
zeros = (abs(df) <= zero_tol).all(axis=0)
zero_cols = [ii for ii, bb in enumerate(zeros) if bb]
print(zero_cols)
print(f"\n---- there are {len(zero_cols)} columns with all zeros ----\n")
print(df.iloc[:, zero_cols[0]], '\n')

df = df.drop(df.columns[zero_cols], axis=1)
display(df)

[538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572]

---- there are 35 columns with all zeros ----

0     -2.890000e-21
1     -1.770000e-21
2      9.440000e-22
3     -4.480000e-21
4     -1.800000e-21
           ...     
751   -1.700000e-20
752    3.220000e-21
753   -2.950000e-22
754    1.770000e-21
755    1.460000e-20
Name: tqwt_meanValue_dec_1, Length: 756, dtype: float64 



,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,250,0,0.80903,0.56355,0.28385,417,416,0.004627,0.000052,0.00064,...,3.0706,3.0190,3.1212,2.4921,3.5844,3.5400,3.3805,3.2003,6.8671,0
752,250,0,0.16084,0.56499,0.59194,415,413,0.004550,0.000220,0.00143,...,1.9704,1.7451,1.8277,2.4976,5.2981,4.2616,6.3042,10.9058,28.4170,0
753,251,0,0.88389,0.72335,0.46815,381,380,0.005069,0.000103,0.00076,...,51.5607,44.4641,26.1586,6.3076,2.8601,2.5361,3.5377,3.3545,5.0424,0
754,251,0,0.83782,0.74890,0.49823,340,339,0.005679,0.000055,0.00092,...,19.1607,12.8312,8.9434,2.2044,1.9496,1.9664,2.6801,2.8332,3.7131,0


In [4]:
features = list(df.columns)
features.remove('id')
features.remove('class')

feat_cut_gender = features.copy()
feat_cut_gender.remove('gender')

============================= I will now check the spread of the data =============================

In [5]:
df_PD_zscore = df[df['class'] == 1].apply(zscore)
df_PD_zscore = df_PD_zscore[feat_cut_gender]
display(df_PD_zscore)

print(df_PD_zscore.abs().max(axis=1))

,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,rapJitter,...,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36
0,0.659533,0.081144,0.475504,-0.777756,-0.770751,0.865495,-0.430602,-0.158468,-0.069067,-0.031302,...,-0.418195,-0.541226,-0.606482,-0.580503,-0.477312,-0.407305,-0.520914,-0.838851,-0.901432,-0.511347
1,0.161563,-0.270966,0.230859,-0.846850,-0.839549,0.979490,-0.448608,-0.238158,-0.128397,-0.169047,...,-0.417943,-0.541320,-0.572165,0.252197,0.032644,-0.091093,-0.230628,-0.603946,-0.679839,0.228842
2,0.649994,-0.552803,0.607167,-0.869881,-0.862482,1.027104,-0.465299,-0.303989,-0.183772,-0.123132,...,-0.418459,-0.541157,-0.617624,-0.341999,-0.143563,-0.091543,-0.501118,-0.820820,-0.891547,-0.911209
3,-1.907155,1.259246,0.627798,-1.491726,-1.481669,2.501259,-0.304513,0.537952,1.034469,0.721708,...,-0.204371,-0.474635,-0.573738,-0.521709,0.024683,-0.455433,-0.508073,-0.704934,-0.685016,-0.931135
4,-2.391747,1.275763,0.160489,-0.823818,-0.816616,0.922891,2.962906,0.939866,0.963273,0.877819,...,-0.168647,-0.402831,-0.492928,-0.486755,-0.398074,-0.264859,-0.105157,-0.502519,-0.765169,-0.873130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,-3.410662,-0.294239,0.541898,0.465935,0.456156,-0.616691,1.363489,0.330065,0.017950,0.336020,...,-0.406443,-0.406042,-0.588216,-0.618609,-0.561330,-0.553237,-0.634119,-0.653511,-0.561992,-0.961363
746,0.535230,-0.566617,-0.637663,0.454419,0.456156,-0.601977,1.312885,0.170685,-0.076978,0.170725,...,-0.398437,-0.481860,-0.579435,-0.637661,-0.575098,-0.537083,-0.589014,-0.814496,-0.908452,-0.968311
747,0.653368,1.042875,-0.600077,0.408356,0.410290,-0.560155,-0.499733,-0.647002,-0.594335,-0.508820,...,0.871969,2.944648,2.771714,1.477951,0.033441,-0.223014,0.000235,0.085656,0.180421,-0.574041
748,0.333680,1.124108,-1.091318,0.373809,0.375891,-0.526113,-0.478836,-0.622748,-0.576931,-0.536369,...,-0.312017,1.866188,0.965808,-0.374838,-0.498128,-0.492254,-0.553057,-0.785051,-0.791222,-0.954863


0      3.844061
1      4.274631
2      4.028714
3      6.889467
4      5.837332
         ...   
745    6.486851
746    6.922866
747    3.197611
748    3.359713
749    4.706292
Length: 564, dtype: float64


In [6]:
def zscore_grouped(df: pd.DataFrame, group_col: str, max_id: int) -> pd.DataFrame:
    df_zscore_id = pd.DataFrame(np.zeros((3*max_id, len(df.columns))), index=np.arange(0, 3*max_id), columns=df.columns)
    for group, group_df in df.groupby(group_col):
#         print(group)
#         display(group_df)
        df_tmp = group_df.apply(zscore)
#         display(df_tmp)
#         print('----')
        if group == max_id:
            break
        df_zscore_id.loc[group_df.index] = df_tmp
    return df_zscore_id

In [7]:
%%time
# zscore within each id as a grouping, so id=0 scores within itself and etc

df_zscore_id = zscore_grouped(df, 'id', 10)  # NOTE: the first 10 id's are class=1, should be a good sampling
df_zscore_id = df_zscore_id[feat_cut_gender]
display(df_zscore_id)

print(df_zscore_id.max(axis=1))

,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,rapJitter,...,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36
0,0.727520,1.267406e+00,0.241542,1.372813,1.372813,-1.354857,1.239912,1.261720,1.238577,1.336306e+00,...,0.018697,0.120522,-0.399316,-1.019804,-1.329878,-1.414212,-0.780536,-0.789889,-0.754907,-0.240199
1,-1.414015,-9.032218e-02,-1.327520,-0.392232,-0.392232,0.326326,-0.030921,-0.077644,-0.028149,-1.069045e+00,...,1.215289,-1.280550,1.374567,1.358430,1.081555,0.708616,1.411577,1.410846,1.413111,1.327049
2,0.686495,-1.177083e+00,1.085978,-0.980581,-0.980581,1.028530,-1.208992,-1.184075,-1.210427,-2.672612e-01,...,-1.233986,1.160028,-0.975250,-0.338626,0.248323,0.705596,-0.631041,-0.620957,-0.658204,-1.086851
3,-0.060218,5.856727e-01,0.002300,-1.395704,-1.375868,1.397504,-1.414149,-1.053849,-0.577601,-9.311294e-01,...,-0.112205,-1.161229,-1.334069,-1.198117,1.413776,-1.008938,-1.126935,-0.320679,1.171139,-0.334932
4,-1.193525,8.219465e-01,-1.225893,0.895357,0.971201,-0.511035,0.695400,-0.289808,-0.829137,-4.562534e-01,...,1.276987,1.279657,1.073477,-0.051623,-0.737356,1.362684,1.303408,1.353182,0.100961,1.357367
5,1.253743,-1.407619e+00,1.223593,0.500347,0.404667,-0.886469,0.718750,1.343657,1.406737,1.387383e+00,...,-1.164781,-0.118429,0.260592,1.249740,-0.676420,-0.353746,-0.176473,-1.032503,-1.272100,-1.022436
6,-1.312885,8.972686e-01,1.394221,1.414214,1.414214,-1.388417,0.116268,0.006748,-0.025477,9.502553e-01,...,-1.180944,1.414110,1.410575,1.407879,1.413733,-1.414076,-0.165499,0.825441,0.326414,1.349653
7,1.111690,4.980345e-01,-0.491902,-0.707107,-0.707107,0.927068,1.162464,1.221357,1.237285,4.319342e-01,...,-0.083344,-0.692267,-0.617488,-0.588146,-0.674953,0.724118,1.299079,0.581757,1.028469,-0.309003
8,0.201194,-1.395303e+00,-0.902319,-0.707107,-0.707107,0.461350,-1.278733,-1.228105,-1.211808,-1.382189e+00,...,1.264288,-0.721843,-0.793087,-0.819733,-0.738781,0.689958,-1.133580,-1.407198,-1.354882,-1.040650
9,1.277935,-9.923389e-02,0.160917,0.000000,0.000000,-0.081683,-0.250183,-0.116248,-0.146446,2.672612e-01,...,1.053903,0.106983,1.413091,-0.144431,0.188873,-0.038627,1.388647,1.412543,1.392191,1.061357


0     1.414214
1     1.414213
2     1.414211
3     1.414213
4     1.414213
5     1.414214
6     1.414214
7     1.414198
8     1.414213
9     1.414213
10    1.414214
11    1.414213
12    1.414214
13    1.414212
14    1.414186
15    1.414210
16    1.414211
17    1.414213
18    1.414209
19    1.414213
20    1.414214
21    1.414207
22    1.414192
23    1.414214
24    1.414212
25    1.414213
26    1.414214
27    1.414199
28    1.414212
29    1.414214
dtype: float64
CPU times: user 9.91 s, sys: 89.7 ms, total: 10 s
Wall time: 10.3 s


In [8]:
# now average by 'id'

df_avg = pd.concat([df[df['id'] == 0].mean(), df[df['id'] == 1].mean()], axis=1).T
for ii in range(2, int(len(df)/3)):
    df_avg.loc[len(df_avg)] = df[df['id'] == ii].mean()
df_avg['id'] = df_avg['id'].astype(int)
df_avg['gender'] = df_avg['gender'].astype(int)
df_avg['class'] = df_avg['class'].astype(int)

In [10]:
print(f"there are {list(df_avg['gender']).count(0)} women and {list(df_avg['gender']).count(1)} men")
print(f"there are {list(df_avg['class']).count(0)} HC and {list(df_avg['class']).count(1)} PD")

there are 122 women and 130 men
there are 64 HC and 188 PD


In [11]:
df_avg_PD = df_avg[df_avg['class'] == 1]
df_avg_PD_zscore = df_avg_PD.apply(zscore)
display(df_avg_PD_zscore)

print(abs(df_avg_PD_zscore).max(axis=1))

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,-1.695609,0.870063,0.689257,-0.261196,0.475768,-0.872428,-0.864731,1.030359,-0.606619,-0.256703,...,-0.666524,-0.790555,-0.291297,-0.249425,-0.240421,-0.514196,-0.974102,-1.055430,-0.471797,NaN
1,-1.681568,-1.149342,-2.644513,1.285504,0.681224,-1.138244,-1.145417,1.448110,2.552551,1.200661,...,-0.542439,-0.698399,-0.632541,-0.322043,-0.457693,-0.395596,-0.860000,-0.989434,-1.090437,NaN
2,-1.667527,0.870063,0.514181,-1.471695,0.095181,0.142505,0.145739,-0.343011,-0.552063,-0.160222,...,0.796081,0.538460,-0.164698,-0.526863,-0.606679,-0.619549,-0.573688,-0.403214,-1.054139,NaN
3,-1.653486,-1.149342,0.732754,-1.374574,0.230666,2.240841,2.234847,-1.683548,-0.662359,-0.716255,...,-0.662239,-0.835275,-0.813530,-0.653732,-0.618437,-0.739054,-1.039122,-1.066584,-0.478088,NaN
4,-1.639445,-1.149342,0.753848,1.054241,1.775731,0.666082,0.667014,-0.607059,2.861564,0.364073,...,-0.572527,-0.723812,-0.807395,-0.698812,-0.660170,-0.758459,-0.721912,-0.276677,0.893138,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,1.688290,-1.149342,0.660260,-2.485682,-0.754768,2.216675,2.210788,-1.655698,0.133455,-0.650242,...,1.654873,1.061729,0.045592,-0.327207,-0.265963,-0.195230,-0.293274,-0.649710,-0.936975,NaN
243,1.716372,0.870063,0.585722,0.846479,-1.314753,0.359991,0.362269,-0.541341,-0.629508,-0.579151,...,0.420064,2.784462,2.976461,2.988139,3.068892,3.108796,2.502611,1.947704,1.702973,NaN
247,1.772536,-1.149342,0.103173,-0.966867,0.635519,0.223056,0.221926,-0.424576,-0.291409,0.211736,...,-0.010884,-0.217952,-0.225743,0.085814,0.610737,0.497018,0.034011,-0.039533,0.113928,NaN
248,1.786577,0.870063,-1.041088,-0.388631,-0.460945,0.593587,0.590827,-0.740557,0.978446,-0.065011,...,-0.559814,-0.775599,-0.811427,-0.698478,-0.645269,-0.740039,-0.979099,-0.987680,-1.143169,NaN


0       4.232586
1       7.162746
2       4.384753
3       4.399282
4       5.252373
         ...    
241    13.530672
243     4.373291
247     3.276598
248     5.124148
249     3.725720
Length: 188, dtype: float64


From comparing df_PD_zscore, df_zscore_id, and df_avg_PD_zscore, it's clear that within each id, there is not a lot of variation, but between each id there is a a lot of variations. This suggests that I need to aggregate by id somehow! But I'm not convinced that averaging is the right way to do it, because the results from the models is very poor when I do that - ie, the models don't beat the control.

In [ ]:
# df = df_avg
# display(df)

============================= I will now perform a correlational analysis =============================

In [ ]:
df_features = df[features]
pearson_all = df_features.corr(method='pearson')
display(pearson_all)

In [ ]:
ax = sns.heatmap(pearson_all.abs(), cmap='magma', xticklabels=False, yticklabels=False)
ax.set_title("Abs(all Pearson correlations)")

In [ ]:
# find all the features which correlate to each other, and drop them
# https://www.projectpro.io/recipes/drop-out-highly-correlated-features-in-python

high_corr_thresh = 0.95

# get the upper right triangle (not including diagonal)
upper_tri = pearson_all.where(np.triu(np.ones(pearson_all.shape), k=1).astype(bool))
display(upper_tri)

# find columns to drop
to_drop = [col for col in upper_tri.columns if any(abs(upper_tri[col]) > high_corr_thresh)]
print(to_drop)
print(f"\n---- number of features to drop due to high correlation = {len(to_drop)} ----\n")

# convert column names to indices and drop
to_drop_ind = [ii for ii in range(len(pearson_all.columns)) if pearson_all.columns[ii] in to_drop]
print(to_drop_ind)
print(f"\n---- number of features to drop due to high correlation = {len(to_drop_ind)} ----\n")
print(len(to_drop_ind)/df_features.shape[1])

In [ ]:
# remove highly correlated features and SAVE the df

orig_num_features = df.shape[1]
df = df.drop(df.columns[to_drop_ind], axis=1)

display(df)
print(df.shape)
print(orig_num_features - len(to_drop))

In [ ]:
ax = sns.heatmap(df.corr(method='pearson').abs(), cmap='magma', xticklabels=False, yticklabels=False)
ax.set_title("Abs(Pearson corr \ highly corr)")

In [ ]:
pearson = df.corr(method='pearson')['class']
print(pearson)
print(type(pearson))

In [ ]:
# low_corr_cutoff = 0.003  # 80, 89, 84, 86 [log regr - accuracy, precision, recall, f1]
# low_corr_cutoff = 0.01   # 81, 90, 84, 87
low_corr_cutoff = 0.03   # 81, 89, 85, 87
# low_corr_cutoff = 0.1    # 82, 92, 83, 87 (slow)
# low_corr_cutoff = 0.2    # 82, 91, 83, 87 (slow)
# low_corr_cutoff = 0.25   # 78, 90, 79, 84 (slow)
# low_corr_cutoff = 0.3    # 75, 88, 78, 83

omit_these_features = pearson[abs(pearson) <= low_corr_cutoff].index
omit_these_features = list(omit_these_features)
# omit_these_features.remove('id')
print(omit_these_features, '\n')
print(len(omit_these_features), len(omit_these_features)/(df.shape[1] - 2))

I will remove features which have a "(very) low" correlation with the class. A weak correlation is generally considered to be less than 0.3, but if I set corr_cutoff to 0.3 I lose 704 of the 753 features. So, through some trial and error, I set an appropriate corr_cutoff.

In [ ]:
features = list(df.columns)
features.remove('id')
features.remove('class')

remaining_features = features.copy()
for ff in omit_these_features:
#     print(ff)
    remaining_features.remove(ff)
print(len(remaining_features))

In [ ]:
# reduce and SAVE the df

df = df[['id'] + remaining_features + ['class']]
display(df)
df.to_csv(data_dir + '/pd_speech_features_reduced.csv', index=None)

In [ ]:
ax = sns.heatmap(df.corr(method='pearson').abs(), cmap='magma', xticklabels=False, yticklabels=False)
ax.set_title("Abs(Pearson corr \ highly corr \ low-class corr)")

I will now check the spearman correlation coefficient, which captures a sense of non-linear correlations, and then compare them to the pearson correlation coefficients.

In [ ]:
spearman = df.corr(method='spearman')['class']
print(spearman)
print(type(spearman))

In [ ]:
high_corr_features = [ff for ff in remaining_features if abs(pearson[ff]) >= 0.3]
print(high_corr_features)
print(len(high_corr_features))

In [ ]:
print("feature,    pearson,    spearman")
print("--------------------------------")
for ff in high_corr_features:
    print(f"{ff},    {pearson[ff]},    {spearman[ff]}")

We see that the following features have a non-linear correlation with the class (diff > 0.05):
[insert].

And the following features are very linearily related to the class (diff < 0.005):
[insert].

============================= I will now perform a PCA to check things out =============================

In [ ]:
class RunPCA():
    def __init__(self, X: np.ndarray, y: pd.Series) -> None:
        self.X = X
        self.y = y
    
    def scale_X(self) -> None:
        sc = StandardScaler()
        self.X = sc.fit_transform(self.X)
    
    def exec_PCA(self) -> None:
        self.pca = PCA()
        self.X = self.pca.fit_transform(self.X)
    
    def set_ev(self) -> None:
        self.explained_variance = self.pca.explained_variance_ratio_
        self.num_components = len(self.explained_variance)
        print(f"number of components = {self.num_components}")
#         print(f"explained_variance = {self.explained_variance}")
        
        sum_cev = 0
        self.cummulative_ev = []
        for ii in range(self.num_components):
            sum_cev += self.explained_variance[ii]
            self.cummulative_ev.append(sum_cev)
#         print(f"cummulative_ev = {self.cummulative_ev}")
    
    def plot_ev(self) -> None:
        plt.figure()
        plt.bar(range(1, self.num_components+1), self.explained_variance)
        plt.title('explained variance')
        plt.xlabel('component')
        plt.ylabel('percent explained')
        plt.show()
        plt.close()
    
    def plot_cev(self) -> None:
        plt.figure()
        plt.bar(range(1, self.num_components+1), self.cummulative_ev)
        plt.title('cummulative explained variance')
        plt.xlabel('component')
        plt.ylabel('percent explained')
        plt.show()
        plt.close()
    
    def plot_PCA(self) -> None:        
        plt.figure()
        plt.scatter(self.X[:, 0], self.X[:, 1], c=self.y)
        plt.title('PC1 vs PC2')
        plt.xlabel(f"PC1 = {100*self.explained_variance[0]:.2f}%")
        plt.ylabel(f"PC2 = {100*self.explained_variance[1]:.2f}%")
        plt.colorbar()
        plt.show()
        plt.close()
        
        plt.figure()
        plt.scatter(self.X[:, 1], self.X[:, 2], c=self.y)
        plt.title('PC2 vs PC3')
        plt.xlabel(f"PC1 = {100*self.explained_variance[1]:.2f}%")
        plt.ylabel(f"PC2 = {100*self.explained_variance[2]:.2f}%")
        plt.colorbar()
        plt.show()
        plt.close()
    
    def run(self) -> None:
        self.scale_X()
        self.exec_PCA()
        self.set_ev()
        self.plot_ev()
        self.plot_cev()
        self.plot_PCA()

In [ ]:
X = df[remaining_features].values
old_X = X.copy()
print(X)
y = df['class']

In [ ]:
rpca = RunPCA(X, y)
rpca.run()

print(X)
print((X == old_X).all())
display(df)

X_pca = rpca.X
print(X_pca)


# NOTE: from running below alone, I learnt that X cannot have all zero columns and highly correlated columns
# X = np.random.rand(10, 6)
# # X = np.concatenate([X, np.array([0,0,0,0,0,0,0,0,0,0]).reshape(-1, 1)], axis=1)
# X = np.concatenate([X, 10*X[:, 2].reshape(-1, 1)], axis=1)
# print(X)
# H = X@np.linalg.pinv(X.T@X)@X.T
# Hii = np.diag(H)
# print(2*X.shape[1]/X.shape[0])
# print(2*Hii.mean())

In PC1 vs PC2 we can see a separation of the classes, which is hopeful!

============================= I will now run a UMAP embedding to check things out =============================

In [ ]:
reducer = umap.UMAP(n_components=2, random_state=42)
embeddings = reducer.fit_transform(X)  # run UMAP on the original X, not the X_pca
# embeddings = reducer.fit_transform(X_pca)

In [ ]:
plt.figure()
plt.scatter(embeddings[:, 0],embeddings[:, 1], c=y)
plt.xlabel('UMAP-1')
plt.ylabel('UMAP-2')
plt.colorbar()
plt.show()
plt.close()

============================= I will now find and remove some outliers =============================

I will remove outliers based on high "leverage" - which is the deviation of the sample in the feature space compared to the general clustering of the samples. They can be calculated via $H_{ii} = [X(X^TX)^{-1}X^T]_{ii}$ for the i-th sample, and "high" happens for $H_{ii} > 2\bar{H_{ii}} = 2p/n$.

"leverage" $H_{ii} = [X(X^TX)^{-1}X^T]_{ii}$,          $H_{ii} > 2\bar{H_{ii}} = 2p/n^*$

In [ ]:
X = X_pca.copy()

print(X)
print(X.shape)
H = X@np.linalg.pinv(X.T@X)@X.T
print(H.shape)
Hii = np.diag(H)
print(len(Hii))

In [ ]:
out_thresh_pn = 2*X.shape[1]/X.shape[0]
out_thresh_hii = 2*Hii.mean()
print(f"the threshold for high is: 2p/n = 2*{X.shape[1]}/{X.shape[0]} = {out_thresh_pn} == {out_thresh_hii} = 2*mean(H_ii)")

Note that in the above cell, if I don't use X_pca (ie, I note out 'X = X_pca.copy()' two cells above) and I just use the original X, then $2p/n \neq 2\bar{H_{ii}}$. I figured out that X cannot have all zero columns, and it cannot have highly correlated columns. If I remove highly correlated columns with high_corr_thresh > 0.5, then I roughly remedy things ($2p/n \approx 2\bar{H_{ii}}$). But for now, we'll just reassign via the PCA and hope that captures the outliers.

In [ ]:
for ii in range(len(Hii)):
    if Hii[ii] >= 1*out_thresh_pn:
#     if Hii[ii] >= 3*out_thresh_hii:
#     if True:
        print(ii, Hii[ii])

In [ ]:
# check = out_thresh_pn
check = 3*out_thresh_hii

ind = list(np.where(Hii > check)[0])
print(type(ind))
print("indices:", ind)
print("classes:", [y[ii] for ii in ind])
print(len(ind), len(ind)/len(Hii))

If there was a weird amount of hits for the condition that $H_{ii} > 2\bar{H_{ii}} = 2p/n$, then I tweaked the threashold to remove <~ 1% of the points, ie, the top 1% of the highest leverage points.

In [ ]:
# remove outliers and SAVE the df

df_outrmv = df.drop(index=ind).copy()
display(df_outrmv)
df_outrmv.to_csv(data_dir + '/pd_speech_features_reduced_outrmv.csv', index=None)
y = df_outrmv['class']

Now redo the PCA and UMAP to see what changed...

In [ ]:
X = df_outrmv.values.copy()  # if you don't add .copy() changing X will change df_outrmv [objects, you see]
print(X)
rpca = RunPCA(X, y)
rpca.run()

X_pca = rpca.X

In [ ]:
reducer = umap.UMAP(n_components=2, random_state=42)
embeddings = reducer.fit_transform(X)  # run UMAP on the original X, not the X_pca
# embeddings = reducer.fit_transform(X_pca)

plt.figure()
plt.scatter(embeddings[:, 0],embeddings[:, 1], c=y)
plt.xlabel('UMAP-1')
plt.ylabel('UMAP-2')
plt.colorbar()
plt.show()
plt.close()

============================= I will now balance the data for the neural net =============================

In [ ]:
df_HC = df_outrmv[df_outrmv['class'] == 0]
df_PD = df_outrmv[df_outrmv['class'] == 1]

print(f"original num of HC =", len(df_HC))
print(f"original num of PD =", len(df_PD))

df_balanced = pd.concat([df_outrmv, df_HC.sample(n=len(df_PD)-len(df_HC), replace=True)]).reset_index(drop=True)
display(df_balanced)

print(f"num of HC =", len(df_balanced[df_balanced['class'] == 0]))
print(f"num of PD =", len(df_balanced[df_balanced['class'] == 1]))

In [ ]:
# SAVE the df from above

df_balanced.to_csv(data_dir + '/pd_speech_features_reduced_outrmv_balanced.csv', index=None)

In [ ]:
# F- I-- N---